In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import sklearn
import pathlib
import os
import torch
import torchvision

In [6]:
from pathlib import Path

positive_directory = Path('data/Positive')
negative_directory = Path('data/Negative')
uncertain_directory = Path('data/Uncertain')

positive_labels_file = "data/Positive/Positive.xlsx"
negative_labels_file = "data/Negative/Negative.xlsx"
uncertain_labels_file = "data/Uncertain/Uncertain.xlsx"

In [7]:
label_map = {'A': 'Positive', 'B': 'Positive', 'C': 'Negative', 'D': 'Uncertain', 'E': 'Uncertain'}
label_to_idx = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}

X = []
y = []

In [8]:
datasets = [
    (positive_labels_file, positive_directory),
    (negative_labels_file, negative_directory),
    (uncertain_labels_file, uncertain_directory),
]

total_images = 0
for labels_file, directory in datasets:
    total_images += len(list(directory.glob('*.jpg')))

img_height, img_width = 224, 224
X_tensor = torch.empty((total_images, 3, img_height, img_width), dtype=torch.float32)
y_tensor = torch.empty(total_images, dtype=torch.long)

idx = 0
for labels_file, directory in datasets:
    labels_df = pd.read_excel(labels_file)
    label_dict = dict(zip(labels_df['fileName'], labels_df['Label'])) 
    
    for file in directory.glob('*.jpg'):
        if file.is_file():
            #print(f"Processing file: {file.name}")
            img = cv2.imread(str(file))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
            img = cv2.resize(img, (img_width, img_height))  
            
            img_tensor = torch.from_numpy(img).float() / 255.0  
            img_tensor = img_tensor.permute(2, 0, 1)  
            
            X_tensor[idx] = img_tensor
            label_letter = label_dict.get(file.name, 'E')  
            y_tensor[idx] = label_to_idx[label_letter]
            
            idx += 1
            #print(idx, file.name, label_letter, y_tensor[idx-1].item())

print(f"Loaded {idx} images into tensors")
print(f"X shape: {X_tensor.shape}, y shape: {y_tensor.shape}")

Loaded 1500 images into tensors
X shape: torch.Size([1500, 3, 224, 224]), y shape: torch.Size([1500])


In [9]:
import torch
from torch import nn

import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [11]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision import datasets

In [12]:
# from sklearn.model_selection import train_test_split

# X_train, y_train, X_test, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2)

In [14]:
import torch.nn as nn
import torch.nn.functional as F

class BasicCNN(nn.Module):
    def __init__(self, num_classes):
        super(BasicCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))    # [B, 16, 112, 112]
        x = self.pool(F.relu(self.conv2(x)))    # [B, 32, 56, 56]
        x = x.view(x.size(0), -1)               # flatten
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [15]:
X_train.size(), y_train.size(), X_test.size(), y_test.size()

(torch.Size([1200, 3, 224, 224]),
 torch.Size([1200]),
 torch.Size([300, 3, 224, 224]),
 torch.Size([300]))

In [16]:
from torch.utils.data import TensorDataset, DataLoader

num_classes = 5
dataset = TensorDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Model setup
model = BasicCNN(num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in range(10):
    running_loss = 0.0
    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/10], Loss: {running_loss:.4f}")


Epoch [1/10], Loss: 61.2381
Epoch [2/10], Loss: 39.1642
Epoch [3/10], Loss: 31.8685
Epoch [4/10], Loss: 29.0210
Epoch [5/10], Loss: 27.2691
Epoch [6/10], Loss: 24.2538
Epoch [7/10], Loss: 22.5591
Epoch [8/10], Loss: 21.0327
Epoch [9/10], Loss: 19.1636
Epoch [10/10], Loss: 17.2415


In [17]:
def evaluate_accuracy(model, X_test, y_test, batch_size=32):
    model.eval()  
    correct = 0
    total = 0

    test_dataset = TensorDataset(X_test, y_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size)

    with torch.no_grad():  
        for X_batch, y_batch in test_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            outputs = model(X_batch)  
            _, predicted = torch.max(outputs, 1)  
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")
    return accuracy


In [18]:
evaluate_accuracy(model, X_test, y_test)

Test Accuracy: 70.67%


70.66666666666667

In [19]:
img_height, img_width = 224, 224  # Set your target dimensions

def test_from_image(filepath):
    img = cv2.imread(str(file))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    img = cv2.resize(img, (img_width, img_height))  # Resize
            
    # Convert directly to tensor
    img_tensor = torch.from_numpy(img).float() / 255.0  # Normalize to [0,1]
    img_tensor = img_tensor.permute(2, 0, 1)  # Change HWC to CHW

    img_tensor = img_tensor.unsqueeze(0).to(device)  # Add batch dimension and move to device
    model.eval()  # Set model to evaluation mode
    with torch.no_grad():
        output = model(img_tensor)  # Forward pass
        _, predicted = torch.max(output, 1)  # Get predicted class index
    
    #print filename and predicted label
    print(f"File: {filepath}, Predicted Label Index: {predicted.item()}, Predicted Label: {list(label_to_idx.keys())[predicted.item()]}")
    return list(label_to_idx.keys())[predicted.item()]


   


